In [122]:
import pandas as pd
import numpy as np
import re
import spacy
import pkg_resources
from spacy.lang.en.stop_words import STOP_WORDS
from collections import Counter, defaultdict

import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import words, wordnet, brown

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
reviews = pd.read_csv("/content/drive/MyDrive/RMP/scraped_comments_with_professor.csv").sample(n=220000, random_state=1)
# reviews[['comment_id', 'firstName', 'lastName', 'prof_class', 'comment', 'clarityRating', 'helpfulRating']].to_csv("/content/drive/MyDrive/RMP/scraped_comments_sample.csv")

# Helper Functions/Comment ____

In [4]:
#@title Dropping nulls
print("Shape before dropping:", reviews.shape)
reviews.drop_duplicates(subset="comment_id", keep="first", inplace=True)

# drop rows containing only "No Comments" (default value assigned by RMP to a review that didn't enter a comment)
reviews = reviews[reviews["comment"] != "No Comments"]

# drop rows containing NaN comment
reviews.dropna(subset=["comment"], inplace=True)

# fill null names with empty string
reviews['firstName'].fillna('', inplace=True)
reviews['lastName'].fillna('', inplace=True)

# Dropping qualityRating == 3.0
reviews['qualityRating'] = (reviews['helpfulRating']+reviews['clarityRating'])/2.0
reviews = reviews[reviews["qualityRating"] != 3.0]
reviews["sentiment"] = reviews["qualityRating"] > 3.0

print("Shape after dropping:", reviews.shape)
reviews.reset_index(drop=True, inplace=True)

Shape before dropping: (500000, 17)
Shape after dropping: (462284, 19)


In [5]:
review_names = reviews.dropna(subset=["firstName", "lastName", "comment", "prof_class"]).loc[:,['firstName', 'lastName', 'comment', 'prof_class']]

In [6]:
#@title Emoticon Mapping
EMOTICONS = {
    u"<3": "emopos",
    u":‑)":"emopos",
    u":-))":"emopos",
    u":-)))":"emopos",
    u":)":"emopos",
    u":))":"emopos",
    u":)))":"emopos",
    u":-]":"emopos",
    u":]":"emopos",
    u":-3":"emopos",
    u":3":"emopos",
    u":->":"emopos",
    u":>":"emopos",
    u"8-)":"emopos",
    u":-}":"emopos",
    u":}":"emopos",
    u":-)":"emopos",
    u":c)":"emopos",
    u":^)":"emopos",
    u"=]":"emopos",
    u"=)":"emopos",
    u":‑D":"emopos",
    u":D":"emopos",
    u"8‑D":"emopos",
    u"8D":"emopos",
    u"X‑D":"emopos",
    u"XD":"emopos",
    u"=D":"emopos",
    u"=3":"emopos",
    u"B^D":"emopos",
    u":-))":"emopos",
    u":-(":"emoneg",
    u":‑(":"emoneg",
    u":(":"emoneg",
    u":‑c":"emoneg",
    u":c":"emoneg",
    u":‑<":"emoneg",
    u":<":"emoneg",
    u":‑[":"emoneg",
    u":[":"emoneg",
    u":-||":"emoneg",
    u">:[":"emoneg",
    u":{":"emoneg",
    u">:(":"emoneg",
    u":'‑(":"emoneg",
    u":'(":"emoneg",
    u":'‑)":"emopos",
    u":')":"emopos",
    u"D‑':":"emoneg",
    u"D:<":"emoneg",
    u"D:":"emoneg",
    u"D8":"emoneg",
    u"D;":"emoneg",
    u"D=":"emoneg",
    u"DX":"emoneg",
    u";‑)":"emopos",
    u";)":"emopos",
    u"*-)":"emopos",
    u"*)":"emopos",
    u";‑]":"emopos",
    u";]":"emopos",
    u";^)":"emopos",
    u":‑,":"emopos",
    u";D":"emopos",
    u":‑P":"emopos",
    u":P":"emopos",
    u"X‑P":"emopos",
    u"XP":"emopos",
    u":‑Þ":"emopos",
    u":Þ":"emopos",
    u"=p":"emopos",
    u":‑/":"emoneg",
    u":/":"emoneg",
    u":-[.]":"emoneg",
    u">:[(\)]":"emoneg",
    u">:/":"emoneg",
    u":[(\)]":"emoneg",
    u"=/":"emoneg",
    u"=[(\)]":"emoneg",
    u":L":"emoneg",
    u"=L":"emoneg",
    u":‑|":"emoneg",
    u":|":"emoneg",
    u"O:‑)":"emopos",
    u"O:)":"emopos",
    u"0:‑3":"emopos",
    u"0:3":"emopos",
    u"0:‑)":"emopos",
    u"0:)":"emopos",
    u":‑b":"emopos",
    u"(>_<)":"emoneg",
    u"(>_<)>":"emoneg",
    u"^_^":"emopos",
    u"(^_^)/":"emopos",
    u"(^O^)／":"emopos",
    u"(^o^)／":"emopos",
    u"('_')":"emoneg",
    u"(/_;)":"emoneg",
    u"(T_T) (;_;)":"emoneg",
    u"(;_;":"emoneg",
    u"(;_:)":"emoneg",
    u"(;O;)":"emoneg",
    u"(:_;)":"emoneg",
    u"(ToT)":"emoneg",
    u";_;":"emoneg",
    u";-;":"emoneg",
    u";n;":"emoneg",
    u"Q.Q":"emoneg",
    u"T.T":"emoneg",
    u"Q_Q":"emoneg",
    u"(-.-)":"emopos",
    u"(-_-)":"emopos",
    u"(；一_一)":"emopos",
    u"(=_=)":"emoneg",
    u"^m^":"emopos",
    u">^_^<":"emopos",
    u"<^!^>":"emopos",
    u"^/^":"emopos",
    u"（*^_^*）" :"emopos",
    u"(^<^) (^.^)":"emopos",
    u"(^^)":"emopos",
    u"(^.^)":"emopos",
    u"(^_^.)":"emopos",
    u"(^_^)":"emopos",
    u"(^^)":"emopos",
    u"(^J^)":"emopos",
    u"(*^.^*)":"emopos",
    u"(^—^）":"emopos",
    u"(#^.^#)":"emopos",
    u"(*^0^*)":"emopos",
    u"(*^^)v":"emopos",
    u"(^_^)v":"emopos",
    u'(-"-)':"emoneg",
    u"(ーー;)":"emoneg",
    u"(＾ｖ＾)":"emopos",
    u"(＾ｕ＾)":"emopos",
    u"(^)o(^)":"emopos",
    u"(^O^)":"emopos",
    u"(^o^)":"emopos",
    u")^o^(":"emopos",
    u":O o_O":"emoneg",
    u"o_0":"emoneg",
    u"o.O":"emoneg",
    u"(o.o)":"emoneg",
    u"(*￣m￣)": "emoneg",
}

for emote, val in EMOTICONS.items():
  EMOTICONS[emote] = val.lower().replace(',', ' ').replace(' ', '_')

In [79]:
#@title Contraction Mapping
contraction_mapping = {
    "dont": "do not",
    "doesnt": "does not",
    "arent": "are not",
    "cant": "can not",
    "couldve": "could have",
    "couldnt": "could not",
    "didnt": "did not",
    "aint": "is not",
    "arent": "are not",
    "hes": "he is",
    "shes": "she is",
    "havent": "have not",
    "hasnt": "has not",
    'youll': "you will",
    "ive": "i have",
    "youve": "you have",
    "shouldve": "should have",
    "im": "i am",
    "isnt": "is not",
    "ain't": "is not", 
    "aren't": "are not",
    "can't": "cannot",
    "'cause": "because", 
    "could've": "could have", 
    "couldn't": "could not", 
    "didn't": "did not",  
    "doesn't": "does not", 
    "don't": "do not", 
    "hadn't": "had not", 
    "hasn't": "has not", 
    "shouldnt": "should not",
    "haven't": "have not", 
    "he'd": "he would",
    "he'll": "he will", 
    "he's": "he is", 
    "how'd": "how did", 
    "how'd'y": "how do you", 
    "how'll": "how will", 
    "how's": "how is",
    "i'd": "i would", 
    "i'd've": "i would have", 
    "i'll": "i will",  
    "i'll've": "i will have",
    "i'm": "i am", 
    "i've": "i have", 
    "isn't": "is not", 
    "it'd": "it would", 
    "it'd've": "it would have", 
    "it'll": "it will", 
    "it'll've": "it will have",
    "it's": "it is", 
    "let's": "let us", 
    "ma'am": "madam", 
    "mayn't": "may not", 
    "might've": "might have",
    "mightn't": "might not",
    "mightn't've": "might not have", 
    "must've": "must have", 
    "mustn't": "must not", 
    "mustn't've": "must not have", 
    "needn't": "need not", 
    "needn't've": "need not have",
    "o'clock": "of the clock", 
    "oughtn't": "ought not", 
    "oughtn't've": "ought not have", 
    "shan't": "shall not", 
    "sha'n't": "shall not", 
    "shan't've": "shall not have", 
    "she'd": "she would", 
    "she'd've": "she would have", 
    "she'll": "she will", 
    "she'll've": "she will have", 
    "she's": "she is", 
    "should've": "should have", 
    "shouldn't": "should not", 
    "shouldn't've": "should not have", 
    "this's": "this is",
    "this'll": "this will",
    "thisll": "this will",
    "that'd": "that would", 
    "that'd've": "that would have", 
    "that's": "that is", 
    "thats": "that is",
    "there'd": "there would", 
    "there'd've": "there would have", 
    "there's": "there is", 
    "theres": "there is",
    "here's": "here is",
    "heres": "here is",
    "they'd": "they would", 
    "they'd've": "they would have", 
    "they'll": "they will", 
    "they'll've": "they will have", 
    "they're": "they are", 
    "they've": "they have", 
    "to've": "to have", 
    "wasn't": "was not", 
    "wasnt": "was not",
    "we'd": "we would", 
    "we'd've": "we would have", 
    "we'll": "we will", 
    "we'll've": "we will have", 
    "we're": "we are", 
    "we've": "we have", 
    "weve": "we have",
    "werent": "were not",
    "weren't": "were not", 
    "what'll": "what will",
    "whatll": "what will",
    "what'll've": "what will have", 
    "what're": "what are",  
    "what's": "what is", 
    "what's": "what is",
    "whatve": "what have",
    "what've": "what have", 
    "when's": "when is", 
    "when've": "when have", 
    "where'd": "where did", 
    "where's": "where is", 
    "wheres": "where is", 
    "where've": "where have",
    "who'll": "who will", 
    "who'll've": "who will have", 
    "who's": "who is", 
    "who've": "who have", 
    "whys": "why is",
    "why's": "why is", 
    "why've": "why have", 
    "will've": "will have", 
    "willve": "will have",
    "won't": "will not", 
    "wont": "will not",
    "won't've": "will not have", 
    "would've": "would have", 
    "wouldve": "would have",
    "wouldn't": "would not", 
    "wouldnt": "would not",
    "wouldn't've": "would not have", 
    "y'all": "you all", 
    "y'all'd": "you all would",
    "y'all'd've": "you all would have",
    "y'all're": "you all are",
    "y'all've": "you all have",
    "you'd": "you would", 
    "you'd've": "you would have", 
    "you'll": "you will", 
    "you'll've": "you will have", 
    "you're": "you are", 
    "you've": "you have"
}

len(contraction_mapping)

147

In [8]:
#@title Vocab Mapping
vocab_mapping = {
    'ta': 'teaching assistant',
    'biz': 'business',
    'hw': 'homework',
    'hws': 'homeworks',
    'faq': 'frequently answered question',
    'faqs': 'frequently answered questions',
    'mcq': 'multiple choice question',
    'mcqs': 'multiple choice questions',
    'frq': 'free response question',
    'frqs': 'free response questions',
    'ppt': 'powerpoint',
    'ppts': 'powerpoints',
    'ques': 'question',
    'bs': 'bullshit',
    'bsing': 'bullshitting',
    'bsed': 'bullshitted',
    'lol': 'laugh out loud',
    'btw': 'by the way',
    'imo': 'in my opinion',
    'imho': 'in my honest opinion',
    'tbh': 'to be honest',
    'asap': 'as soon as possible',
    'idc': 'i do not care',
    'omg': 'oh my god',
    'ppl': 'people',
    'rip': 'rest in peace',
    'srsly': 'seriously',
    'thx': 'thanks',
    'txt': 'text',
    'ur': 'your',
    'tho': 'though',
    'wtf': 'what the fuck',
    'wth': 'what the heck',
    'bc': 'because',
    'b4': 'before',
    'h8': 'hate',
    'jk': 'just kidding',
    'cuz': 'because'
}

## Stopwords

In [9]:
general_stopwords = set(["mr", "ms", "dr", "s", "t", "i", "me", "myself", "is", "the"])

gen_stops = set(["mr", "ms", "dr", "doctor", "s", "t", "i", "me", "myself", "is", "she", "he", "we", "him", "her", "it"])
domain_stops = set(["book", "books", "college", "colleges", "lecture", "lectures", "university", "universities", "lab", "labs", "hw", "hws", "quiz", "quizzes", "prof", "professor", "teacher", "class", "classes", "course", "courses"])
stopwords = gen_stops.union(domain_stops)

## Helper Functions

In [10]:
def remove_urls(text):
    return re.sub(r'https?://\S+|www\.\S+', ' ', text)

def remove_phones(text):
    return re.sub(r'\d{3}-\d{3}-\d{4}', ' ', text)

def remove_emails(text):
    return re.sub(r'[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+', ' ', text)

def remove_html_entities(text):
  text = re.sub('&[0-9a-zA-Z#]+;', ' ', text)
  return re.sub('&#63;?', '', text)

def remove_html_tags(text):
  return re.sub('<.{1,6}?>', ' ', text)

def convert_emoticons(text):
  return EMOTICONS.get(text, text)

def expand_contraction(text): # Before expanding contraction, might want to clean of symbols that are not '
  return contraction_mapping.get(text, text)

# Preprocessing Pipeline

In [80]:
from collections import defaultdict

all_words = defaultdict(int)

def preprocess_pipeline(df, func=None):
  cnt = 0 # to keep track of progress
  comments_proper = []
  for index, review in df.iterrows():
    comment = review['comment']
    fname = review['firstName'].lower().split(' ')
    lname = review['lastName'].lower().split(' ')
    names = set(fname + lname)

    cnt += 1
    if cnt % 25000 == 0:
      print(cnt)

    comment = remove_urls(comment)
    comment = remove_phones(comment)
    comment = remove_emails(comment)
    comment = remove_html_entities(comment)
    comment = remove_html_tags(comment)

    comment_split = comment.split(' ')
    new_comment_split = []
    for i, word in enumerate(comment_split):
      word = convert_emoticons(word)
      word = word.lower()
      word = expand_contraction(word)
      word = re.sub("[^a-z\s]+", ' ', word)   # replace characters that are not alphabetic, space, or underscore
      word = re.sub(r'(.)\1\1+', '\g<1>', word)  # replace any three character+ sequence with one
      word = re.sub('\s+', ' ', word)
      word = word.strip() # trailing whitespace because punctuation replaced by space
      new_comment_split.extend(word.split(' '))

    for i, word in enumerate(new_comment_split):
      if func and func(word, names):
        new_comment_split[i] = ''
      else:
        new_comment_split[i] = vocab_mapping.get(word, word)

    comment = ' '.join(new_comment_split)
    comment = re.sub('\s+', ' ', comment)
    comment = comment.strip()

    # comment = ' '.join(word for word in comment.split() if len(word) > 1)

    comments_proper.append(comment)
    for word in comment.split(' '):
      all_words[word] += 1
    # spellchecked_comments.append(' '.join(sym_spell.lookup(word, Verbosity.TOP, max_edit_distance=2, include_unknown=True)[0].term for word in comment.split()))
  return comments_proper

In [81]:
comments_proper = preprocess_pipeline(reviews)
# print(len(all_names)) # 49421 before
# print(len(all_words)) # 72182 words

reviews['cleanedComment'] = comments_proper

25000
50000
75000
100000
125000
150000
175000
200000
225000
250000
275000
300000
325000
350000
375000
400000
425000
450000


In [63]:
all_names = set()
for index, review in reviews.iterrows():
  fname = review['firstName'].lower().split(' ')
  lname = review['lastName'].lower().split(' ')
  names = set(fname + lname)
  all_names.update(names)
print(len(all_words))
print(len(all_names))

86807
53788


AttributeError: ignored

In [90]:
all_words = defaultdict(int)
reviews['cleanedComment'] = preprocess_pipeline(reviews, lambda x, nms: x in nms)

25000
50000
75000
100000
125000
150000
175000
200000
225000
250000
275000
300000
325000
350000
375000
400000
425000
450000


In [69]:
all_words_set = set(all_words.keys())
print(len(all_words_set))
print(len(all_names))
print(len(all_words_set.intersection(all_names))) # Cut a lot of features

69427
53788
6686


In [75]:
for comment in reviews.loc[100:200, 'cleanedComment']:
  print(comment)
  print()

absolutely fantastic life changing

bad teacher very harsh grader details are all or nothing

professor josh is awesome he is very friendly and has excellent dancing skills he makes things very clear and is very helpful outside of class take your math classes with him

he is good teacher he like teaching and wants to teach he lets you revise your papers

she is nice but not good teacher she is not help me at all very lazy the class was a waste of money and time do not take her class

amazing teacher

she was by far my favorite teacher that i have ever had she was so helpful i had her for psych and she made it so interesting i would recomend her to anyone

easy class show up and do the work all the test questions come from the mcgraw hill homework online if you do all the homework you get for homework grade just do the work

is a wonderful teacher with a lot of knowledge of criminology i found her classes to be very interesting come to class pay attention and read the textbook and you w

## Abbreviation and Stuff???

In [124]:
cleaned_comments = reviews['cleanedComment']

alphabet = set(list("abcdefghijklmnopqrstuvwxyz"))
words_avoid = set(["all", "up", "is", "so", "to", "be", "for", "on", "he", "his", "the", "had", "of", "and", "are",
                   "dr", "she", "her", "oh", "try", "you", "she", "now", "not", "my", "but", "an", "wow", "do", "if", "fun",
                   "was", "in", "at", "one", "can", "how", "in", "was", "lot", "get", "it", "put", "any", "got", "our", "him",
                   "guy", "am", "law", "end", "we", "us", "use", "ask", "rid", "max", "joy", "no", "did", "sad", "day",
                   "fan", "few", "off", "era", "top", "big", "red", "hot", "age", "pie", "age", "by", "mad", "gut", "fed",
                   "two", "see", "air", "hug", "bag", "cut", "has", "or", "buy", "sit", "way", "lay", "say", "ok", "mrs",
                   "shy", "god", "pay", "kid", "key", "too", "why", "box", "aid", "ton", "art", "met", "dry", "let", "bet",
                   "far", "sum", "bad", "yet", "yes", "ass", "set", "boy", "map", "low", "old", "who", "lab", "bam",
                   "pop", "sob", "dog", "run", "hat", "lie", "ego", "own", "its", "go", "add", "pet", "job", "due", "may", "nap", "ink",
                   "new", "bit", "as", "out", "me", "mr", "man", "ms", "fix"])
words_avoid.update(alphabet)
words_all_3 = set()
for index, review in reviews.sample(1000, random_state=1).iterrows():
  words_less_3 = set(word for word in review['cleanedComment'].split(' ') if len(word) <= 3 and word not in words_avoid)
  if words_less_3:
    words_all_3.update(words_less_3)

print(words_all_3)
print(len(words_all_3))

{'min', '', 'hof', 'im', 'er', 'que', 'ova', 'bon', 'esl', 'pro', 'crj', 'cmu', 'gr', 'dq', 'de', 'ee', 'co', 'th', 'ic', 'pg', 'fix', 'non', 'ken', 'jc', 'acc', 'ob', 'gcc', 'lin', 'via', 'dra', 'psu', 'hep', 'sur', 'mid', 'ho', 'mat', 'csi', 'bas', 'wte', 'ear', 'dat', 'sw', 'phd', 'um', 'isn', 'wa', 'wen', 'civ', 'nor', 'wsu', 'ffs', 'wk', 'meh', 'cuz', 'psy', 'ie', 'rel', 'alg', 'ccq', 'ge', 'hes', 'def', 'gsi', 'se', 'ucf', 'ce', 'hip', 'prf', 'per', 'dep', 'un', 'ver', 'amc', 'huh', 'ben', 'ol', 'bb', 'als', 'pre', 'duh', 'jen', 'ace', 'exp', 'la', 'et', 'hez', 'yu', 'hum', 'fsc', 'don', 'le', 'won', 'eco', 'lap', 'ucr', 'lit', 'env', 'pp', 'ed', 'hae', 'fav', 'soc', 'bmw', 'ad', 'usc', 'bro', 'abt', 'son', 'qs', 'pts', 'org', 'ugh', 'cam', 'wiz', 'wcd', 'ec', 'bio', 'neg', 'int', 'cs', 'mag', 'cps', 'lil', 'ii', 'bcc', 'etc', 'gov', 'est', 'hcc', 'mo', 'req', 'act', 'eng', 'nd', 'hm', 'din', 'tas', 'sth', 'col', 'sat', 'swc', 'com', 'mis', 'oop', 're', 'web'}
146


In [126]:
counter_3_sample = Counter()
for comment in reviews.loc[500:1000, 'cleanedComment']:
  cond = False
  cleaned_word_set = set(comment.split(' '))
  for word in cleaned_word_set:
    if word in words_all_3:
      cond = True
      counter_3_sample[word] += 1
  if cond:
    print(comment)
    print()

print(counter_3_sample)

# To Remove:
'''
bio, etc, eng, mails
'''
# Abbreviations / Shortened
'''
etc, def, ed, via, pts, pg, tas(teaching assistants?), phd, req, bro, abt, org, gen, ed(unsure??), lil, fav
'''

this class has a horrifying amount of work you get tested twice per topic before class and during class every class period tuesday and thursday while she means well and knows her stuff she expects way too much for a gen ed requirement only take this class if love groupwork every week and if you are willing to have it take over your life

i liked her i know some students struggled but i am not sure why the tests were of average difficulty if you study and use apr frequently then you will pass i never went in for office hours and i do not think they were essential she makes class non stressful and interesting she only holds class on friday as needed which is appreciated

she does not know what she is teaching every five seconds she stops to stare at the board and asks what to do because she herself is lost unorganized in grading she made a mistake on my test and did not even bring the grade sheet to fix it for another month if you ask her a question she will say you do this and not clari

'\netc, def, ed, via, pts, pg, tas(teaching assistants?), phd, req, bro, abt, org\n'

In [14]:
# !pip install wordsegment

In [15]:
# Might need to spellcheck comments before segmenting them

# from wordsegment import load, segment
# load()
# for comment in comments_proper[:200]:
#   new_comment = segment(comment)
#   if comment.split(' ') != new_comment:
#     print(comment.split(' '))
#     print(new_comment)
#     print('\n')

# Pycontractions
Very long to do as opposed to a simple contraction mapping, will hold off on this for now.

In [16]:
# !python --version

Python 3.7.13


In [35]:
# Because of weird java dependencies this needs to be done
# !sudo apt-get update -y

# !sudo apt install openjdk-8-jdk
# !sudo update-alternatives --set java /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java
# !java -version

Ign:1 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Hit:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:4 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Packages [912 kB]
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:13 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [91.1 kB]
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:14 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:15 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelea

In [18]:
# %pip install git+https://github.com/MCFreddie777/language-check.git
# %pip install pycontractions

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/MCFreddie777/language-check.git to /tmp/pip-req-build-saqqgiwj
  Running command git clone -q https://github.com/MCFreddie777/language-check.git /tmp/pip-req-build-saqqgiwj
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [25]:
# from pycontractions import Contractions
# help(Contractions)

Help on class Contractions in module pycontractions.contractions:

class Contractions(builtins.object)
 |  Contractions(w2v_path=None, lang_code='en-US', kv_model=None, api_key=None)
 |  
 |  Expand and contract common English contractions in text.
 |  
 |  Uses a combination of pattern replacement, grammar checking, and Word Mover's Distance.
 |  
 |  Methods defined here:
 |  
 |  __init__(self, w2v_path=None, lang_code='en-US', kv_model=None, api_key=None)
 |      w2v_path is a path to an embedding model used for calculating the Word Mover's Distance.
 |  
 |  contract_texts(self, texts)
 |      Return a generator over an iterable of text where each result has contracted common expansions.
 |  
 |  expand_texts(self, texts, precise=False, scores=False)
 |      Return a generator over an iterable of text where each result has common contractions expanded.
 |      
 |      If precise == True then it will use a much slower method that does not assume all occurrences
 |      of the same

In [31]:
# import gensim.downloader as api
# model = api.load("glove-twitter-100")

In [37]:
# cont = Contractions(kv_model=model)
# cont.load_models() # prevents loading the model on the first expand_texts

In [39]:
# Non-precise
# print(list(cont.expand_texts(["you ain't anything special, she's much faster and has Jamie's dog"])))
# list(cont.expand_texts(["I'd like to know how I'd done that!",
#                             "We're going to the zoo and I don't think I'll be home for dinner.",
#                             "Theyre going to the zoo and she'll be home for dinner."]))

["you have not anything special, she has much faster and has Jamie's dog"]


['I had like to know how I had done that!',
 'we are going to the zoo and I do not think I will be home for dinner.',
 'they are going to the zoo and she will be home for dinner.']

In [40]:
# Precise
# print(list(cont.expand_texts(["you ain't anything special, she's much faster and has Jamie's dog"], precise=True)))
# list(cont.expand_texts(["I'd like to know how I'd done that!",
#                             "We're going to the zoo and I don't think I'll be home for dinner.",
#                             "Theyre going to the zoo and she'll be home for dinner."], precise=True))

["you have not anything special, she has much faster and has Jamie's dog"]


['I would like to know how I had done that!',
 'we are going to the zoo and I do not think I will be home for dinner.',
 'they are going to the zoo and she will be home for dinner.']

In [49]:
# list(cont.expand_texts(["dont you see youre a tool! testing to see if this'll catch contractions that have no apostrophe and have mispellings", "he'll not like what he sees. hell, I don't know what I'm seeing. you think she'll like it?"], precise=True))

["do not you see you are a tool! testing to see if this'll catch contractions that have no apostrophe and have mispellings",
 'he will not like what he sees. hell, I do not know what I am seeing. you think she will like it?']

In [54]:
# One sample
# list(cont.expand_texts(["this'll be a decent test."]))

["this'll be a decent test."]

In [58]:
# def preprocess_pipeline2(df, print_index=25000):
#   cnt = 0 # to keep track of progress
#   comments_proper = []
#   for index, review in df.iterrows():
#     comment = review['comment']
#     fname = review['firstName'].lower().split(' ')
#     lname = review['lastName'].lower().split(' ')
#     names = set(fname + lname)
#     all_names.update(names)

#     cnt += 1
#     if cnt % print_index == 0:
#       print(cnt)

#     comment = remove_urls(comment)
#     comment = remove_phones(comment)
#     comment = remove_emails(comment)
#     comment = remove_html_entities(comment)
#     comment = remove_html_tags(comment)

#     comment = list(cont.expand_texts([comment], precise=True))[0]
#     comment_split = comment.split(' ')
#     new_comment_split = []
#     for i, word in enumerate(comment_split):
#       word = convert_emoticons(word)
#       word = word.lower()
#       # word = expand_contraction(word)
#       word = re.sub("[^a-z\s]+", ' ', word)   # replace characters that are not alphabetic, space, or underscore
#       word = re.sub(r'(.)\1\1+', '\g<1>', word)  # replace any three character+ sequence with one
#       word = re.sub('\s+', ' ', word)
#       word = word.strip() # trailing whitespace because punctuation replaced by space
#       new_comment_split.extend(word.split(' '))

#     # for i, word in enumerate(new_comment_split):
#     #   if word in names or word in stopwords:
#     #     new_comment_split[i] = ''
#     #   else:
#     #     new_comment_split[i] = vocab_mapping.get(word, word)

#     for i, word in enumerate(new_comment_split):
#       new_comment_split[i] = vocab_mapping.get(word, word)

#     comment = ' '.join(new_comment_split)
#     comment = re.sub('\s+', ' ', comment)
#     comment = comment.strip()

#     # comment = ' '.join(word for word in comment.split() if len(word) > 1)

#     comments_proper.append(comment)
#     for word in comment.split(' '):
#       all_words[word] += 1
#     # spellchecked_comments.append(' '.join(sym_spell.lookup(word, Verbosity.TOP, max_edit_distance=2, include_unknown=True)[0].term for word in comment.split()))
#   return comments_proper

# comments_proper_contractions = preprocess_pipeline2(reviews.loc[:25000], print_index=1000)

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000


In [61]:
for comm1, comm2 in zip(comments_proper[500:600], comments_proper_contractions[500:600]):
  if comm1 != comm2:
    print(comm1)
    print(comm2)
    print('\n')

definately on the easy end of cal classes lets you make a cheat sheet for every test and you can cram anything you want into one side of a full sheet of paper you can use every previous one plus and extra one on the final if you do any homework you can do her tests
definately on the easy end of cal classes let us you make a cheat sheet for every test and you can cram anything you want into one side of a full sheet of paper you can use every previous one plus and extra one on the final if you do any homework you can do her tests


the sweets old man ever he lets you resubmit assignments you get low grades on if you attend class and pay attention you should have no problem passing his quizzes he gives alot of them but you have to be willing to put in work it is not a easy a
the sweets old man ever he let us you resubmit assignments you get low grades on if you attend class and pay attention you should have no problem passing his quizzes he gives alot of them but you have to be willing to